In [10]:
from sympy import Add, Mul, symbols, sympify, I, ccode


In [11]:
complex_expr = sympify("sin(z + I) / sin(z**3 - 1)")

In [12]:
def handle_complex_operations(expr, depth=0):
    """
    Recursively handle complex operations by breaking them down into equivalent sequences 
    of real arithmetic operations and calls to helper functions.
    """
    x, y = symbols('x y', real=True)
    z = x + I*y

    # Base case: if the expression is atomic (i.e., cannot be broken down further)
    if expr.is_Atom:
        return ccode(expr).replace("I", "1")  # Replacing I representation

    # Recursive case
    refined_parts = []
    for part in expr.args:
        refined_parts.append(handle_complex_operations(part, depth + 1))

    # Replacement with helper functions
    if expr.func == Add:
        # Handle addition
        real_parts = [part for part in refined_parts if "I" not in part]
        imag_parts = [part.replace('*I', '') for part in refined_parts if "I" in part]
        
        real_str = ' + '.join(real_parts) if real_parts else '0'
        imag_str = ' + '.join(imag_parts) if imag_parts else '0'
        return f"complex_add_real({real_str}, {imag_str}, 0, 0)"
    
    elif expr.func == Mul:
        # Handle multiplication
        real_parts = [part for part in refined_parts if "I" not in part]
        imag_parts = [part.replace('*I', '') for part in refined_parts if "I" in part]
        
        real_str = ' * '.join(real_parts) if real_parts else '1'
        imag_str = ' * '.join(imag_parts) if imag_parts else '1'
        
        return (f"complex_mul_real({real_str}, {imag_str}, 0, 0), "
                f"complex_mul_imag({real_str}, {imag_str}, 0, 0)")
    
    elif expr.func == Pow and '-1' in ccode(expr.args[1]):
        # Handle division
        numerator = refined_parts[0]
        denominator = refined_parts[1].replace('pow(', '').replace(', -1)', '')
        
        return (f"complex_div_real({numerator}, 0, {denominator}, 0), "
                f"complex_div_imag({numerator}, 0, {denominator}, 0)")
    
    else:
        # For other operations, just join the parts with the function name
        return f"{expr.func.__name__}({', '.join(refined_parts)})"

# Test the function with the complex expression again after the refinement
handled_complex_expr = handle_complex_operations(complex_expr)
handled_complex_expr

'complex_mul_real(complex_div_real(sin(complex_add_real(-1 + Pow(z, 3), 0, 0, 0)), 0, -1, 0), complex_div_imag(sin(complex_add_real(-1 + Pow(z, 3), 0, 0, 0)), 0, -1, 0) * sin(complex_add_real(1 + z, 0, 0, 0)), 1, 0, 0), complex_mul_imag(complex_div_real(sin(complex_add_real(-1 + Pow(z, 3), 0, 0, 0)), 0, -1, 0), complex_div_imag(sin(complex_add_real(-1 + Pow(z, 3), 0, 0, 0)), 0, -1, 0) * sin(complex_add_real(1 + z, 0, 0, 0)), 1, 0, 0)'